# Partie 2 : Classification basée sur des descripteurs de haut niveau ( Kammoun Med Habib & Dhia Elhak Toukebri)

In [ ]:
import tensorflow as tf

# Configuration GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limiter la croissance de la mémoire GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Vérification de la disponibilité du GPU
print("Devices GPU disponibles :")
print(tf.config.list_physical_devices('GPU'))

Devices GPU disponibles :
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.applications.resnet import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os


## 1.Détection de caractéristiques spatiales avec des réseaux de neurones profonds

In [ ]:
def extract_cnn_features(image_path, model, preprocess_func, resize_dim=(224, 224)):
    """
    Extrait les caractéristiques CNN d'une image en utilisant un modèle préentraîné.

    Args:
        image_path (str): Chemin de l'image.
        model (tf.keras.Model): Modèle CNN préentraîné.
        preprocess_func (callable): Fonction de prétraitement du modèle.
        resize_dim (tuple): Dimensions pour redimensionner l'image (par défaut (224, 224)).

    Returns:
        np.array: Vecteur de caractéristiques.
    """
    # Chargement et redimensionnement de l'image
    image = load_img(image_path, target_size=resize_dim)
    image_array = img_to_array(image)
    # Prétraitement de l'image
    image_preprocessed = preprocess_func(image_array)
    image_preprocessed = np.expand_dims(image_preprocessed, axis=0)  # Ajouter une dimension batch
    # Extraire les caractéristiques
    features = model.predict(image_preprocessed, verbose=0)
    return features.flatten()


## 2.Chargement des modéles VGG16 et Resnet50

In [ ]:
# Charger le modèle ResNet50 sans les couches FC
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Charger le modèle VGG16 sans les couches FC
vgg_model = VGG16(weights='imagenet', include_top=False, pooling='avg')


## 3.Fonction d' extration des caractéristiques

In [ ]:
import tensorflow as tf
import os
import numpy as np

def process_cnn_dataset(folder_path, model, preprocess_func, resize_dim=(224, 224)):
    features, labels = [], []

    # Compter le nombre total d'images
    total_images = 0
    for label_folder in ["O", "R"]:
        full_path = os.path.join(folder_path, label_folder)
        total_images += len(os.listdir(full_path))

    # Compteur pour suivre la progression
    processed_images = 0

    # Utilisation explicite du GPU
    with tf.device('/GPU:0'):
        for label_folder, label in [("O", 0), ("R", 1)]:
            full_path = os.path.join(folder_path, label_folder)
            for image_name in os.listdir(full_path):
                image_path = os.path.join(full_path, image_name)

                # Extraction des caractéristiques
                feature = extract_cnn_features(image_path, model, preprocess_func, resize_dim)
                features.append(feature)
                labels.append(label)

                # Mise à jour et affichage de la progression
                processed_images += 1
                progress = (processed_images / total_images) * 100
                print(f"[{progress:.2f}%] Extraction de caractéristiques pour {image_name} sur : {tf.test.gpu_device_name()} ({processed_images}/{total_images})", end='\r')

        # Saut de ligne après la progression
        print()

    # Conversion en tableau NumPy sur GPU si possible
    features_array = tf.convert_to_tensor(features)
    labels_array = tf.convert_to_tensor(labels)

    print(f"Calcul effectué sur : {tf.test.gpu_device_name()}")
    return np.array(features_array), np.array(labels_array)

# Configuration GPU avant l'exécution
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU disponible : {gpus[0].name}")
    except RuntimeError as e:
        print(e)

train_folder = "C:/Users/touke/Downloads/g_dechets/g_dechets/Train"  # Mettez ici le chemin vers votre dossier principal

GPU disponible : /physical_device:GPU:0




```
# Ce texte est au format code
```

## 4. Extraction avec vgg16

### 4.1 Extraction des données du Train

In [ ]:
X_train_cnn1, y_train_cnn1 = process_cnn_dataset(train_folder, vgg_model, preprocess_vgg)
print(f"Caractéristiques CNN extraites : {X_train_cnn1.shape}")

Extraction de caractéristiques pour O_10000.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10001.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10003.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10004.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10005.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10007.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10009.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10010.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10011.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10012.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10013.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10014.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10015.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10016.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10017.jpg 

### 4.2 sauvgarder les caractéristiques de train

In [ ]:
####Enregistrer les caractéristiques de train
import pandas as pd
df_vgg = pd.DataFrame(X_train_cnn1)
df_vgg.to_csv("X_train_vgg(train).csv ", index=False)
# Convertir les labels en DataFrame
df_labels = pd.DataFrame(y_train_cnn1, columns=['Label'])
df_labels.to_csv("y_train_vgg.csv", index=False)

In [ ]:
print(f"Caractéristiques CNN extraites : {X_train_cnn1.shape}")

Caractéristiques CNN extraites : (11816, 512)


### 4.3 Extraction du données de test

In [ ]:
Test_folder="C:/Users/touke/Downloads/g_dechets/g_dechets/Test"
X_test_cnn1, y_test_cnn1 = process_cnn_dataset(Test_folder, vgg_model, preprocess_vgg)


Extraction de caractéristiques pour O_1.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10041.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10042.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10043.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10044.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10045.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10046.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10047.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10048.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10049.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10050.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10051.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10052.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10053.jpg sur : /

In [ ]:
print(f"Caractéristiques CNN extraites test : {X_test_cnn1.shape}")

Caractéristiques CNN extraites test : (5064, 512)


### 4.4 sauvgarder les caractéristiques de test

In [ ]:
import pandas as pd
####Enregistrer les caractéristiques de test
df_labels = pd.DataFrame(y_test_cnn1, columns=['Label'])
df_labels.to_csv("y_test_vgg.csv", index=False)

VGG_df1 = pd.DataFrame(X_test_cnn1)
VGG_df1.to_csv('features_cnn_VGG (x_test).csv', index=False)

<ul><li><strong>Entrée de l'image</strong> : L'image est redimensionnée à 224x224 pixels et prétraitée pour correspondre aux attentes du modèle.</li><li><strong>Couches convolutives</strong> :<ul><li>Les premières couches détectent des caractéristiques simples (bords, motifs, textures).</li><li>Les couches profondes apprennent des caractéristiques plus complexes et abstraites (formes, structures).</li></ul></li><li><strong>Activation ReLU</strong> : Une fonction d'activation est appliquée après chaque convolution pour ajouter de la non-linéarité.</li><li><strong>Max Pooling</strong> : Réduction de la taille des cartes de caractéristiques tout en conservant les informations essentielles.</li><li><strong>Global Average Pooling (GAP)</strong> : Les cartes de caractéristiques finales sont moyennées pour produire un vecteur de taille fixe.</li><li><strong>Vecteur de caractéristiques (512 dimensions)</strong> : Ce vecteur compact représente les caractéristiques extraites de l'image, prêt pour des tâches comme la classification ou d'autres analyses.</li></ul>

In [ ]:
df=pd.read_csv('X_train_vgg(train).csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11816 entries, 0 to 11815
Columns: 512 entries, 0 to 511
dtypes: float64(512)
memory usage: 46.2 MB


## 5. Extraction avec *Resnet50*

### 5.1Extraction du données de train

In [ ]:
X_train_cnn, y_train_cnn = process_cnn_dataset(train_folder, resnet_model, preprocess_resnet)  # Mettez ici le chemin vers votre dossier principal

Extraction de caractéristiques pour O_10000.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10001.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10003.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10004.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10005.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10007.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10009.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10010.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10011.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10012.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10013.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10014.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10015.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10016.jpg sur : /device:GPU:0
Extraction de caractéristiques pour O_10017.jpg 

In [ ]:
print(f"Caractéristiques CNN extraites : {X_train_cnn.shape}")

Caractéristiques CNN extraites : (11816, 2048)


### 5.2 Extraction du données de test

In [ ]:
X_test_cnn, y_test_cnn = process_cnn_dataset(Test_folder, resnet_model, preprocess_resnet)  # Mettez ici le chemin vers votre dossier principal
print(f"Caractéristiques CNN extraites : {X_test_cnn.shape}")

[100.00%] Extraction de caractéristiques pour R_9999.jpg sur : /device:GPU:0 (5064/5064)
Calcul effectué sur : /device:GPU:0
Caractéristiques CNN extraites : (5064, 2048)


### 5.3 sauvgarder les caractéristiques de train et de test

In [ ]:
import pandas as pd
###############Train#########################
# Supposons que X_train_cnn est un tableau NumPy (11816, 2048)
features_df_resnet = pd.DataFrame(X_train_cnn)  # Convertir en DataFrame
features_df_resnet.to_csv('features_cnn_resnet(X_train).csv', index=False)  # Sauvegarde en CSV
import pandas as pd
# Convertir les labels en DataFrame
df_labels = pd.DataFrame(y_train_cnn, columns=['Label'])
df_labels.to_csv("y_train_resnet.csv", index=False)

In [ ]:
##################Test#######################
features1_df_resnet = pd.DataFrame(X_test_cnn)  # Convertir en DataFrame
features1_df_resnet.to_csv('features_cnn_resnet(x_test).csv', index=False)  # Sauvegarde en CSV
import pandas as pd
# Convertir les labels en DataFrame
df1_labels = pd.DataFrame(y_test_cnn, columns=['Label'])
df1_labels.to_csv("y_test_resnet.csv", index=False)

In [ ]:
features_df_resnet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11816 entries, 0 to 11815
Columns: 2048 entries, 0 to 2047
dtypes: float32(2048)
memory usage: 92.3 MB


<ul><li><strong>Entrée de l'image</strong> : L'image passe d'abord par le modèle ResNet.</li><li><strong>Couches convolutives</strong> : Ces couches apprennent des caractéristiques à différents niveaux de complexité :<ul><li>Les premières couches détectent des caractéristiques simples (bords, textures).</li><li>Les couches profondes captent des caractéristiques plus abstraites représentant des objets ou des parties d'objets.</li></ul></li><li><strong>Couches résiduelles</strong> : ResNet utilise des blocs résiduels pour éviter la disparition du gradient et permettre l'apprentissage dans des réseaux très profonds.</li><li><strong>Global Average Pooling (GAP)</strong> : À la fin du réseau, les cartes de caractéristiques (ex. <span class="katex"><span class="katex-mathml"><math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>H</mi><mo>×</mo><mi>W</mi><mo>×</mo><mi>C</mi></mrow><annotation encoding="application/x-tex">H \times W \times C</annotation></semantics></math></span><span class="katex-html" aria-hidden="true"><span class="base"><span class="strut" style="height: 0.7667em; vertical-align: -0.0833em;"></span><span class="mord mathnormal" style="margin-right: 0.08125em;">H</span><span class="mspace" style="margin-right: 0.2222em;"></span><span class="mbin">×</span><span class="mspace" style="margin-right: 0.2222em;"></span></span><span class="base"><span class="strut" style="height: 0.7667em; vertical-align: -0.0833em;"></span><span class="mord mathnormal" style="margin-right: 0.13889em;">W</span><span class="mspace" style="margin-right: 0.2222em;"></span><span class="mbin">×</span><span class="mspace" style="margin-right: 0.2222em;"></span></span><span class="base"><span class="strut" style="height: 0.6833em;"></span><span class="mord mathnormal" style="margin-right: 0.07153em;">C</span></span></span></span>) sont réduites par GAP pour obtenir un vecteur de caractéristiques de taille fixe (2048).</li><li><strong>Vecteur de caractéristiques (2048 dimensions)</strong> : Ce vecteur résume l'image dans un espace de caractéristiques de haut niveau, prêt à être utilisé pour des tâches comme la classification ou l'apprentissage automatique.</li></ul>

### 6. Apprentissage Automatique en utilisant les caractéristiques extrairés

---

par VGG16 et ResNet

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Entraîner un modèle
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
print("=====modéle svm pour RESNET=====")
# Modèle SVM
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train_cnn, y_train_cnn)

# Prédiction et évaluation
y_pred_svm = svm_model.predict(X_test_cnn)
print("Résultats SVM:")
print(f"Accuracy: {accuracy_score(y_test_cnn, y_pred_svm)}")
print(classification_report(y_test_cnn, y_pred_svm))
print("************************************")
print("=====modéle svm pour VGG16=====")
# Modèle SVM
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train_cnn1, y_train_cnn1)

# Prédiction et évaluation
y_pred1_svm = svm_model.predict(X_test_cnn1)
print("Résultats SVM:")
print(f"Accuracy: {accuracy_score(y_test_cnn1, y_pred1_svm)}")
print(classification_report(y_test_cnn1, y_pred1_svm))


=====modéle svm pour RESNET=====
Résultats SVM:
Accuracy: 0.9512243285939969
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2931
           1       0.94      0.94      0.94      2133

    accuracy                           0.95      5064
   macro avg       0.95      0.95      0.95      5064
weighted avg       0.95      0.95      0.95      5064

************************************
=====modéle svm pour VGG16=====
Résultats SVM:
Accuracy: 0.95260663507109
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2931
           1       0.94      0.94      0.94      2133

    accuracy                           0.95      5064
   macro avg       0.95      0.95      0.95      5064
weighted avg       0.95      0.95      0.95      5064



In [ ]:
from sklearn.ensemble import RandomForestClassifier
print("=====modéle rdf pour RESNET=====")
# Modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_cnn, y_train_cnn)

# Prédiction et évaluation
y_pred_rf = rf_model.predict(X_test_cnn)
print("Résultats Random Forest:")
print(f"Accuracy: {accuracy_score(y_test_cnn, y_pred_rf)}")
print(classification_report(y_test_cnn, y_pred_rf))
print("**************************")
print("=====modéle rdf pour VGG16=====")

rf_model.fit(X_train_cnn1, y_train_cnn1)

# Prédiction et évaluation
y_pred1_rf = rf_model.predict(X_test_cnn1)
print("Résultats Random Forest:")
print(f"Accuracy: {accuracy_score(y_test_cnn1, y_pred1_rf)}")
print(classification_report(y_test_cnn1, y_pred1_rf))

=====modéle rdf pour RESNET=====
Résultats Random Forest:
Accuracy: 0.9589257503949447
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      2931
           1       0.97      0.93      0.95      2133

    accuracy                           0.96      5064
   macro avg       0.96      0.96      0.96      5064
weighted avg       0.96      0.96      0.96      5064

**************************
=====modéle rdf pour VGG16=====
Résultats Random Forest:
Accuracy: 0.9514218009478673
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      2931
           1       0.96      0.92      0.94      2133

    accuracy                           0.95      5064
   macro avg       0.95      0.95      0.95      5064
weighted avg       0.95      0.95      0.95      5064



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
print("=====modéle gb pour RESNET=====")
# Modèle Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train_cnn, y_train_cnn)

# Prédiction et évaluation
y_pred_gb = gb_model.predict(X_test_cnn)
print("Résultats Gradient Boosting:")
print(f"Accuracy: {accuracy_score(y_test_cnn, y_pred_gb)}")
print(classification_report(y_test_cnn, y_pred_gb))
print("**************************")
print("=====modéle gb pour VGG16=====")
gb_model.fit(X_train_cnn1, y_train_cnn1)

# Prédiction et évaluation
y_pred1_gb = gb_model.predict(X_test_cnn1)
print("Résultats Gradient Boosting:")
print(f"Accuracy: {accuracy_score(y_test_cnn1, y_pred1_gb)}")
print(classification_report(y_test_cnn1, y_pred1_gb))

=====modéle gb pour RESNET=====
Résultats Gradient Boosting:
Accuracy: 0.9597156398104265
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      2931
           1       0.96      0.95      0.95      2133

    accuracy                           0.96      5064
   macro avg       0.96      0.96      0.96      5064
weighted avg       0.96      0.96      0.96      5064

**************************
=====modéle gb pour VGG16=====
Résultats Gradient Boosting:
Accuracy: 0.9516192733017378
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      2931
           1       0.95      0.94      0.94      2133

    accuracy                           0.95      5064
   macro avg       0.95      0.95      0.95      5064
weighted avg       0.95      0.95      0.95      5064



In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print("=====modéle RESNET=====")
# Comparaison des performances
print(f"{'Modèle':<20} {'Accuracy':<10} {'F1-score':<10}")
print("-" * 40)

# Affichage des résultats pour chaque modèle avec un format propre
print(f"{'SVM':<20} {accuracy_score(y_test_cnn, y_pred_svm):<10.2f} {f1_score(y_test_cnn, y_pred_svm):<10.2f}")
print(f"{'Random Forest':<20} {accuracy_score(y_test_cnn, y_pred_rf):<10.2f} {f1_score(y_test_cnn, y_pred_rf):<10.2f}")
print(f"{'Gradient Boosting':<20} {accuracy_score(y_test_cnn, y_pred_gb):<10.2f} {f1_score(y_test_cnn, y_pred_gb):<10.2f}")
print("**************************")
print("=====modéle VGG16=====")
print(f"{'SVM':<20} {accuracy_score(y_test_cnn1, y_pred1_svm):<10.2f} {f1_score(y_test_cnn1, y_pred1_svm):<10.2f}")
print(f"{'Random Forest':<20} {accuracy_score(y_test_cnn1, y_pred1_rf):<10.2f} {f1_score(y_test_cnn1, y_pred1_rf):<10.2f}")
print(f"{'Gradient Boosting':<20} {accuracy_score(y_test_cnn1, y_pred1_gb):<10.2f} {f1_score(y_test_cnn1, y_pred1_gb):<10.2f}")





=====modéle RESNET=====
Modèle               Accuracy   F1-score  
----------------------------------------
SVM                  0.95       0.94      
Random Forest        0.96       0.95      
Gradient Boosting    0.96       0.95      
**************************
=====modéle VGG16=====
SVM                  0.95       0.94      
Random Forest        0.95       0.94      
Gradient Boosting    0.95       0.94      


## 7. Extraction en utilisant un transformer ViT ( partie faites dans google colab car il nous manque de ressources le fichier Vit.ipynb)

In [ ]:
from transformers import AutoFeatureExtractor, AutoModel
from PIL import Image
import numpy as np
import os
import torch

# Charger le modèle et le préprocesseur
model_name = "google/vit-base-patch16-224"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
transformer_model = AutoModel.from_pretrained(model_name)

# Vérifier la disponibilité du GPU et définir le périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer_model.to(device)

def extract_transformer_features(image_path, feature_extractor, model):
    # Charger et prétraiter l'image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")

    # Déplacer les entrées vers le GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Passer dans le modèle
    with torch.no_grad():
        outputs = model(**inputs)

    # Utiliser la sortie du dernier token CLS pour les caractéristiques globales
    features = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()
    return features.flatten()

def process_transformer_dataset(folder_path, feature_extractor, model):
    features, labels = [], []

    # Compter le nombre total d'images
    total_images = 0
    for label_folder in ["O", "R"]:
        full_path = os.path.join(folder_path, label_folder)
        total_images += len(os.listdir(full_path))

    # Compteur pour suivre la progression
    processed_images = 0

    # Parcours des dossiers d'images
    for label_folder, label in [("O", 0), ("R", 1)]:
        full_path = os.path.join(folder_path, label_folder)
        for image_name in os.listdir(full_path):
            image_path = os.path.join(full_path, image_name)

            # Extraction des caractéristiques
            feature = extract_transformer_features(image_path, feature_extractor, model)
            features.append(feature)
            labels.append(label)

            # Mise à jour et affichage de la progression
            processed_images += 1
            progress = (processed_images / total_images) * 100
            print(f"[{progress:.2f}%] Extraction de caractéristiques pour {image_name} ({processed_images}/{total_images})", end='\r', flush=True)

    # Saut de ligne après la progression
    print()

    # Conversion en tableau NumPy
    features_array = np.array(features)
    labels_array = np.array(labels)

    print(f"Calcul effectué sur : {device}")
    return features_array, labels_array



In [ ]:
# Exemple d'utilisation avec ViT
###Traiin
train_folder = "/content/g_dechets/g_dechets/Train"  # Mettez ici le chemin vers votre dossier principal
##X_train_transformer, y_train_transformer = process_transformer_dataset(train_folder, feature_extractor, transformer_model)

print(f"Caractéristiques Transformer extraites(train) : {X_train_transformer.shape}")



<b>Les Features de transformer ont été extrairé par google collab et enregistré sous forme de fichier csv
vous trouvez ci joint le code pour extraire les caractéristiques à l'aide de ViT Transformer dans le notebook ViT.pynb . </b>

In [ ]:
###Test
test_folder = "/content/g_dechets/g_dechets/Test"  # Mettez ici le chemin vers votre dossier principal
X_test_transformer, y_test_transformer = process_transformer_dataset(test_folder, feature_extractor, transformer_model)

print(f"Caractéristiques Transformer extraites(test) : {X_test_transformer.shape}")

Chargement de données extrairées par ViTransformer

In [ ]:
# Charger les caractéristiques (features) et les labels (labels)
X_train_transformer = pd.read_csv('C:/Users/touke/Downloads/cvision/features/features_cnn_transformer(train).csv').values
y_train_transformer = pd.read_csv('C:/Users/touke/Downloads/cvision/features/y_train_trans.csv')['Label'].values

# Vérifier les formes des données
print("Shape of X_train_transformer:", X_train_transformer.shape)
print("Shape of y_train_transformer:", y_train_transformer.shape)

Shape of X_train_transformer: (11816, 768)
Shape of y_train_transformer: (11816,)


In [ ]:
# Charger les caractéristiques (features) et les labels (labels)
X_test_transformer = pd.read_csv('C:/Users/touke/Downloads/cvision/features/features_cnn_transformer(test).csv').values
y_test_transformer = pd.read_csv('C:/Users/touke/Downloads/cvision/features/y_test_trans.csv')['Label'].values

# Vérifier les formes des données
print("Shape of X_test_transformer:", X_test_transformer.shape)
print("Shape of y_test_transformer:", y_test_transformer.shape)

Shape of X_test_transformer: (5064, 768)
Shape of y_test_transformer: (5064,)




```
# Ce texte est au format code
```

## 8. Apprentissage Automatique en utilisant les caractéristiques extrairés par le Transformer

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Entraîner un modèle
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
print("=====modéle svm pour transformer=====")
# Modèle SVM
svm_vit = SVC(kernel='linear', C=1, random_state=42)
svm_vit.fit(X_train_transformer, y_train_transformer)

# Prédiction et évaluation
y_pred2_svm = svm_vit.predict(X_test_transformer)
print("Résultats SVM:")
print(f"Accuracy: {accuracy_score(y_test_transformer, y_pred2_svm)}")
print(classification_report(y_test_transformer, y_pred2_svm))
print("************************************")

from sklearn.ensemble import RandomForestClassifier
print("=====modéle rf pour transformerT=====")
# Modèle Random Forest
rf_vit = RandomForestClassifier(n_estimators=100, random_state=42)
rf_vit.fit(X_train_transformer, y_train_transformer)

# Prédiction et évaluation
y_pred2_rf = rf_vit.predict(X_test_transformer)
print("Résultats Random Forest:")
print(f"Accuracy: {accuracy_score(y_test_transformer, y_pred2_rf)}")
print(classification_report(y_test_transformer, y_pred2_rf))
print("**************************")




=====modéle svm pour transformer=====
Résultats SVM:
Accuracy: 0.9654423380726699
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2931
           1       0.96      0.96      0.96      2133

    accuracy                           0.97      5064
   macro avg       0.96      0.96      0.96      5064
weighted avg       0.97      0.97      0.97      5064

************************************
=====modéle rf pour transformerT=====
Résultats Random Forest:
Accuracy: 0.9729462875197472
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2931
           1       0.98      0.95      0.97      2133

    accuracy                           0.97      5064
   macro avg       0.97      0.97      0.97      5064
weighted avg       0.97      0.97      0.97      5064

**************************


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
print("=====modéle gb pour transformer=====")
# Modèle Gradient Boosting
gb_vit = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_vit.fit(X_train_transformer, y_train_transformer)

# Prédiction et évaluation
y_pred2_gb = gb_vit.predict(X_test_transformer)
print("Résultats Gradient Boosting:")
print(f"Accuracy: {accuracy_score(y_test_transformer, y_pred2_gb)}")
print(classification_report(y_test_transformer, y_pred2_gb))

print("=====résumé pour transformer=====")
print(f"{'SVM':<20} {accuracy_score(y_test_transformer, y_pred2_svm):<10.2f} {f1_score(y_test_transformer, y_pred2_svm):<10.2f}")
print(f"{'Random Forest':<20} {accuracy_score(y_test_transformer, y_pred2_rf):<10.2f} {f1_score(y_test_transformer, y_pred2_rf):<10.2f}")
print(f"{'Gradient Boosting':<20} {accuracy_score(y_test_transformer, y_pred2_gb):<10.2f} {f1_score(y_test_transformer, y_pred2_gb):<10.2f}")

=====modéle gb pour transformer=====
Résultats Gradient Boosting:
Accuracy: 0.9695892575039494
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      2931
           1       0.97      0.96      0.96      2133

    accuracy                           0.97      5064
   macro avg       0.97      0.97      0.97      5064
weighted avg       0.97      0.97      0.97      5064

=====résumé pour transformer=====
SVM                  0.97       0.96      
Random Forest        0.97       0.97      
Gradient Boosting    0.97       0.96      


## 9.Comparaison des résultats des modéles ML entre les modléles d'extraction vgg16, Resnet et ViT(transformer)

<pre class="font-styrene border-border-100/50 overflow-x-scroll whitespace-nowrap rounded border-[0.5px] shadow-[0_2px_12px_rgba(0,0,0,0.05)]"><table class="bg-bg-100 min-w-full border-separate border-spacing-0 text-sm leading-[1.88888]"><thead class="border-b-border-100/50 border-b-[0.5px] text-left"><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><th class="text-text-000 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] font-400 px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Architecture</th><th class="text-text-000 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] font-400 px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Algorithme</th><th class="text-text-000 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] font-400 px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Accuracy</th><th class="text-text-000 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] font-400 px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">F1-score</th><th class="text-text-000 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] font-400 px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Observations</th></tr></thead><tbody><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">RESNET</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">SVM</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.951</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.94</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Performance standard</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">RESNET</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Random Forest</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.959</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.95</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Légère amélioration</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">RESNET</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Gradient Boosting</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.960</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.95</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Meilleur modèle RESNET</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">VGG16</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">SVM</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.953</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.94</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Performance similaire</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">VGG16</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Random Forest</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.951</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.94</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Stable</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">VGG16</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Gradient Boosting</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.952</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.94</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Cohérent</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Transformer</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">SVM</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.965</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.96</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Nette progression</td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Transformer</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Random Forest</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.973</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.97</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]"><strong>MEILLEUR MODÈLE</strong></td></tr><tr class="[tbody>&amp;]:odd:bg-bg-500/10"><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Transformer ViT</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Gradient Boosting</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.970</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">0.96</td><td class="border-t-border-100/50 [&amp;:not(:first-child)]:-x-[hsla(var(--border-100) / 0.5)] border-t-[0.5px] px-2 [&amp;:not(:first-child)]:border-l-[0.5px]">Très bon résultat</td></tr></tbody></table></pre>

<div data-is-streaming="false" class="group  relative  pt-3.5  pb-[1.125rem]  px-4  relative  rounded-2xl  -tracking-[0.015em]  bg-[linear-gradient(to_bottom,_hsla(var(--bg-000)/0.75)_0%,_hsla(var(--bg-000)_/_0)_90%)]  before:absolute  before:inset-0  before:bg-[radial-gradient(ellipse_at_left_top,_hsla(var(--bg-000)/0.5)_0%,_hsla(var(--bg-000)/0.3)_60%)]  before:rounded-2xl  before:border-[0.5px]  before:border-[hsla(var(--border-100)/0.15)]  before:shadow-[0_4px_24px_rgba(0,0,0,0.015)]  before:[transition:opacity_150ms_ease-out,_transform_250ms_cubic-bezier(0.695,0.555,0.655,1.650)]  before:z-0  before:data-[is-streaming=&quot;true&quot;]:opacity-0  before:data-[is-streaming=&quot;true&quot;]:scale-[0.995]  before:data-[is-streaming=&quot;false&quot;]:pointer-events-none" style="opacity: 1; transform: none;"><div class="font-claude-message  pr-4  md:pr-9  relative  leading-[1.65rem]  [&amp;_pre>div]:bg-bg-300  [&amp;_.ignore-pre-bg>div]:bg-transparent  [&amp;_pre]:-mr-4  md:[&amp;_pre]:-mr-9"><div><div class="grid-cols-1 grid gap-2.5 [&amp;_>_*]:min-w-0"><p class="whitespace-pre-wrap break-words">Analyse Comparative des Techniques d'Extraction de Caractéristiques d'Image</p>
<p class="whitespace-pre-wrap break-words">Principaux Points Clés :</p>
<ol class="-mt-1 [li>&amp;]:mt-2 list-decimal space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">Performance Globale</li>
</ol>
<ul class="-mt-1 [li>&amp;]:mt-2 list-disc space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">Transformer : Meilleure architecture (Accuracy 0.965-0.973)</li>
<li class="whitespace-normal break-words" index="1">RESNET : Performance solide (Accuracy 0.951-0.960)</li>
<li class="whitespace-normal break-words" index="2">VGG16 : Performance stable (Accuracy 0.951-0.953)</li>
</ul>
<ol class="-mt-1 [li>&amp;]:mt-2 list-decimal space-y-2 pl-8" start="2" depth="0">
<li class="whitespace-normal break-words" index="0">Caractéristiques des Architectures</li>
</ol>
<ul class="-mt-1 [li>&amp;]:mt-2 list-disc space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">RESNET : Blocs résiduels, adapté aux réseaux profonds</li>
<li class="whitespace-normal break-words" index="1">VGG16 : Convolutions empilées, extraction hiérarchique</li>
<li class="whitespace-normal break-words" index="2">Transformer : Mécanisme d'attention, capture globale des informations</li>
</ul>
<ol class="-mt-1 [li>&amp;]:mt-2 list-decimal space-y-2 pl-8" start="3" depth="0">
<li class="whitespace-normal break-words" index="0">Algorithmes de Classification</li>
</ol>
<ul class="-mt-1 [li>&amp;]:mt-2 list-disc space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">SVM : Meilleur sur séparation des classes</li>
<li class="whitespace-normal break-words" index="1">Random Forest : Meilleure généralisation (Accuracy max 0.973)</li>
<li class="whitespace-normal break-words" index="2">Gradient Boosting : Correction progressive des erreurs</li>
</ul>
<ol class="-mt-1 [li>&amp;]:mt-2 list-decimal space-y-2 pl-8" start="4" depth="0">
<li class="whitespace-normal break-words" index="0">Combinaison Optimale</li>
</ol>
<ul class="-mt-1 [li>&amp;]:mt-2 list-disc space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">Transformer + Random Forest : Performances supérieures</li>
<li class="whitespace-normal break-words" index="1">Accuracy de 0.973, représentant l'état de l'art actuel</li>
</ul>
<ol class="-mt-1 [li>&amp;]:mt-2 list-decimal space-y-2 pl-8" start="5" depth="0">
<li class="whitespace-normal break-words" index="0">Recommandation Principale</li>
</ol>
<ul class="-mt-1 [li>&amp;]:mt-2 list-disc space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">Privilégier l'architecture Transformer avec l'algorithme Random Forest pour des tâches complexes de classification d'images</li>
</ul></div></div></div><div class="absolute -bottom-0 -right-1.5" style="transform: none;"><div class="border-0.5 border-border-300 flex items-center translate-y-1/2 rounded-lg shadow-sm transition bg-bg-000 p-1"><div class="text-text-400 flex i,1,222,165.29Z"></path></svg></button></div></div></div></div></div>/ul>ransformer</span></code></div>

<ul><li>Le <strong>Transformeur ViT</strong> offre la meilleure performance globale, avec des scores maximaux obtenus par Random Forest et Gradient Boosting (Accuracy et F1-score de 0.97 et 0.96 respectivement).</li><li><strong>ResNet</strong> est particulièrement performant avec SVM, atteignant une Accuracy de 0.96.</li><li><strong>VGG16</strong> fournit des résultats solides, mais légèrement inférieurs à ResNet et Transformeurs.</li></ul>

## 10. Analyse Comparative global avec la partie 1

<table> <thead> <tr> <th><strong>Modèle</strong></th> <th><strong>Caractéristiques</strong></th> <th><strong>Avant Sélection (Initial)</strong></th> <th><strong>Après SelectKBest</strong></th> <th><strong>Après Random Forest</strong></th> </tr> </thead> <tbody> <tr> <td><strong>SVM</strong></td> <td><strong>HOG</strong></td> <td>Accuracy: 0.805<br>F1: 0.80</td> <td>Accuracy: 0.786<br>F1: 0.78</td> <td>Accuracy: 0.772<br>F1: 0.77</td> </tr> <tr> <td></td> <td><strong>LBP</strong></td> <td>Accuracy: 0.7397<br>F1: 0.73</td> <td>Accuracy: 0.7397<br>F1: 0.73</td> <td>Accuracy: 0.7397<br>F1: 0.73</td> </tr> <tr> <td><strong>Random Forest</strong></td> <td><strong>HOG</strong></td> <td>Accuracy: 0.807<br>F1: 0.80</td> <td>Accuracy: 0.789<br>F1: 0.79</td> <td>Accuracy: 0.780<br>F1: 0.78</td> </tr> <tr> <td></td> <td><strong>LBP</strong></td> <td>Accuracy: 0.8059<br>F1: 0.80</td> <td>Accuracy: 0.8059<br>F1: 0.80</td> <td>Accuracy: 0.8051<br>F1: 0.80</td> </tr> <tr> <td><strong>Gradient Boosting</strong></td> <td><strong>HOG</strong></td> <td>Accuracy: 0.814<br>F1: 0.81</td> <td>Accuracy: 0.787<br>F1: 0.79</td> <td>Accuracy: 0.778<br>F1: 0.78</td> </tr> <tr> <td></td> <td><strong>LBP</strong></td> <td>Accuracy: 0.8035<br>F1: 0.80</td> <td>Accuracy: 0.8035<br>F1: 0.80</td> <td>Accuracy: 0.8037<br>F1: 0.80</td> </tr> </tbody> </table>

<strong>Analyse Comparative global</strong>

<ul><li><strong>HOG et LBP (Avant sélection)</strong> offrent des performances faibles par rapport aux autres méthodes, avec des Accuracy ne dépassant pas 0.81.</li><li>L'utilisation de <strong>SelectKBest</strong> améliore significativement les scores pour HOG et LBP, mais reste inférieure aux performances des caractéristiques CNN.</li><li>Les modèles basés sur les <strong>Transformeurs</strong> et ResNet dominent, avec des performances maximales atteignant 0.97 pour Accuracy et 0.96 pour F1-score.</li><li>Les <strong>méthodes CNN (ResNet, VGG16, Transformeur)</strong> surpassent systématiquement les méthodes traditionnelles (HOG, LBP).</li></ul>

<p>Ces résultats montrent que les modèles modernes de deep learning extraient des caractéristiques plus riches et discriminantes que les méthodes traditionnelles.</p>

Le Transformer `ViT` donne la meilleur Performance .

# Enregistrer le meilleur modéle ML pour l'utiliser dans la partie 3

<p class="whitespace-pre-wrap break-words"><b>🏆 Classement Final</b></p>

<ol class="-mt-1 [li>&amp;]:mt-2 list-decimal space-y-2 pl-8" depth="0">
<li class="whitespace-normal break-words" index="0">Transformer + Random Forest (Accuracy 0.973)</li>
<li class="whitespace-normal break-words" index="1">Transformer + Gradient Boosting (Accuracy 0.970)</li>
<li class="whitespace-normal break-words" index="2">RESNET + Gradient Boosting (Accuracy 0.960)</li>
</ol>

In [ ]:
import joblib
joblib.dump(rf_vit, 'best_model.pkl')


['best_model.pkl']